In [127]:
import tensorflow as tf
import numpy as np
from textloader import TextLoader
from tensorflow.python.ops.rnn_cell import RNNCell
#from mygru import mygru

In [128]:


class mygru( RNNCell ):
    def __init__( self, num_units ):
        self.num_units = num_units

    @property
    def state_size(self):
        return self.num_units

    @property
    def output_size(self):
        return self.num_units

    def __call__( self, inputs, state, scope=None ):
        with tf.name_scope("evil_gru") as scope:
            init = tf.contrib.layers.variance_scaling_initializer()
            xt = inputs
            h_minusone = state
            Wz = tf.get_variable("W_z", shape = [inputs.get_shape()[1], state.get_shape()[1]], initializer = init)
            Wh = tf.get_variable("W_h", shape = [inputs.get_shape()[1], state.get_shape()[1]], initializer = init)
            Wr = tf.get_variable("W_r", shape = [inputs.get_shape()[1], state.get_shape()[1]], initializer = init)
            Uz = tf.get_variable("U_z", shape = [state.get_shape()[1], state.get_shape()[1]], initializer = init)
            Ur = tf.get_variable("U_r", shape = [state.get_shape()[1], state.get_shape()[1]], initializer = init)
            Uh = tf.get_variable("U_h", shape = [state.get_shape()[1], state.get_shape()[1]], initializer = init)
            bz = tf.get_variable("b_z", shape = [state.get_shape()[1]], initializer = init)
            br = tf.get_variable("b_r", shape = [state.get_shape()[1]], initializer = init)
            bh = tf.get_variable("b_h", shape = [state.get_shape()[1]], initializer = init)
            zt = tf.sigmoid(tf.nn.bias_add((tf.matmul(xt,Wz) + tf.matmul(h_minusone,Uz)) , bz))
            rt = tf.sigmoid( tf.nn.bias_add((tf.matmul(xt,Wr) + tf.matmul(h_minusone,Ur)) , br))
            h_twid = tf.tanh(tf.nn.bias_add((tf.matmul(xt,Wh) + tf.matmul( tf.multiply(rt, h_minusone), Uh)), bh))
            ht = tf.multiply(zt,h_minusone) + tf.multiply((1-zt),  h_twid)
        return ht, ht

In [129]:

#
# -------------------------------------------
#
# Global variables

batch_size = 50
sample_batch = 1
sequence_length = 50
sample_seq = [1]

data_loader = TextLoader( ".", batch_size, sequence_length )

vocab_size = data_loader.vocab_size  # dimension of one-hot encodings
state_dim = 128

num_layers = 2
c = .0001

tf.reset_default_graph()

#
# ==================================================================
# ==================================================================
# ==================================================================
#

# define placeholders for our inputs.  
# in_ph is assumed to be [batch_size,sequence_length]
# targ_ph is assumed to be [batch_size,sequence_length]

in_ph = tf.placeholder( tf.int32, [ batch_size, sequence_length ], name='inputs' )
targ_ph = tf.placeholder( tf.int32, [ batch_size, sequence_length ], name='targets' )
in_onehot = tf.one_hot( in_ph, vocab_size, name="s_input_onehot" )

inputs = tf.split( in_onehot, sequence_length, axis=1 )
inputs = [ tf.squeeze(input_, [1]) for input_ in inputs ]
targets = tf.split( targ_ph, sequence_length, axis=1 )

s_in_ph = tf.placeholder( tf.int32, sample_seq, name='s_inputs' )
s_in_onehot = tf.one_hot( s_in_ph, vocab_size, name="s_input_onehot" )
s_inputs = tf.split( s_in_onehot, sample_seq, axis=0 )


# ------------------
# YOUR COMPUTATION GRAPH HERE

#comp_graph(batch_size)
R = False
with tf.name_scope("computation_graph"):
    cell0 = mygru( state_dim )
    cell1 = mygru( state_dim )
    multi_cell = tf.contrib.rnn.MultiRNNCell([cell0, cell1])

    initial_state = multi_cell.zero_state(batch_size, tf.float32)
    seq_out, final_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, initial_state, multi_cell)
    initializer = tf.contrib.layers.variance_scaling_initializer()
    logits = []
    for i in xrange(sequence_length):
        logits.append(tf.layers.dense(seq_out[i], vocab_size, reuse = R))
        R = True
    
    weights = [1.0 for _ in xrange(sequence_length)]
    loss =  tf.contrib.legacy_seq2seq.sequence_loss(logits, targets, weights)
    optim = tf.train.AdamOptimizer(learning_rate = c).minimize(loss)


# ------------------
# YOUR SAMPLER GRAPH HERE

#comp_graph(sample_batch, R = True)
R = True
with tf.name_scope("sampler_graph"):
    s_initial_state = multi_cell.zero_state(sample_batch, tf.float32)
    s_seq_out, s_final_state = tf.contrib.legacy_seq2seq.rnn_decoder(s_inputs, s_initial_state, multi_cell)
    #print s_seq_out
    s_initializer = tf.contrib.layers.variance_scaling_initializer()
    s_probs  = [tf.layers.dense(s_seq_out[i], vocab_size, reuse = R) for i in xrange(len(s_seq_out))]



#
# ==================================================================
# ==================================================================
# ==================================================================
#

def sample( num=200, prime='ab' ):

    # prime the pump

    # generate an initial state. this will be a list of states, one for
    # each layer in the multicell.
    s_state = sess.run( s_initial_state )

    # for each character, feed it into the sampler graph and
    # update the state.
    for char in prime[:-1]:
        x = np.ravel( data_loader.vocab[char] ).astype('int32')
        feed = { s_in_ph:x }
        for i, s in enumerate( s_initial_state ):
            feed[s] = s_state[i]
        s_state = sess.run( s_final_state, feed_dict=feed )

    # now we have a primed state vector; we need to start sampling.
    ret = prime
    char = prime[-1]
    for n in range(num):
        x = np.ravel( data_loader.vocab[char] ).astype('int32')

        # plug the most recent character in...
        feed = { s_in_ph: x }
        for i, s in enumerate( s_initial_state ):
            feed[s] = s_state[i]
        ops = [s_probs]
        ops.extend( list(s_final_state) )

        #print x
        retval = sess.run( ops, feed_dict=feed )

        s_probsv = retval[0]
        s_state = retval[1:]
                # ...and get a vector of probabilities out!

        # now sample (or pick the argmax)
        sample = np.argmax( s_probsv[0] )
        #sample = np.random.choice( vocab_size, p=s_probsv[0] )

        pred = data_loader.chars[sample]
        ret += pred
        char = pred

    return ret

#
# ==================================================================
# ==================================================================
# ==================================================================
#

sess = tf.Session()
sess.run( tf.global_variables_initializer() )
summary_writer = tf.summary.FileWriter( "./tf_logs", graph=sess.graph )

lts = []

print "FOUND %d BATCHES" % data_loader.num_batches

for j in range(20):

    state = sess.run( initial_state )
    data_loader.reset_batch_pointer()

    for i in range( data_loader.num_batches ):

        x,y = data_loader.next_batch()

        # we have to feed in the individual states of the MultiRNN cell
        feed = { in_ph: x, targ_ph: y }
        for k, s in enumerate( initial_state ):
            feed[s] = state[k]

        ops = [optim,loss]
        ops.extend( list(final_state) )
                # retval will have at least 3 entries:
        # 0 is None (triggered by the optim op)
        # 1 is the loss
        # 2+ are the new final states of the MultiRNN cell
        retval = sess.run( ops, feed_dict=feed )

        lt = retval[1]
        state = retval[2:]

        if i%1000==0:
            print "%d %d\t%.4f" % ( j, i, lt )
            lts.append( lt )

    print sample( num=60, prime="And " )
#    print sample( num=60, prime="ababab" )
#    print sample( num=60, prime="foo ba" )
#    print sample( num=60, prime="abcdab" )


with open("alma_gru.txt", "w") as f:
    for i in xrange(30):
        f.write(sample( num=60, prime="And " ) + "\n")
        
summary_writer.close()

#
# ==================================================================
# ==================================================================
# ==================================================================
#

#import matplotlib
#import matplotlib.pyplot as plt
#plt.plot( lts )
#plt.show()



loading preprocessed files
FOUND 186 BATCHES
0 0	4.3010
And eo to                                              e    e   
1 0	3.0483
And toe the the the the the the the the the the the the the the 
2 0	2.8049
And toe the the the the the the the the the the the the the the 
3 0	2.5956
And toe the the the the the the the the the the the the the the 
4 0	2.4291
And tore the the the the the the rerere the the rerere the the r
5 0	2.2966
And the the the the the the reand the reath the reathe word the 
6 0	2.1902
And the the the the the reore the reore the reath the reath the 
7 0	2.1034
And the poren the reore the reore the reore the reore the reath 
8 0	2.0291
And the poren the reore the reore the reore the reath the reath 
9 0	1.9630
And the pamant the reath the reore the poren the reath the reath
10 0	1.9036
And the pamanites the Lamanites the Lamanites the reperen the wi
11 0	1.8506
And the pamanites the Lamanites the Lamanites the Lamanites the 
12 0	1.8030
And the pamanites the Lamanit

In [130]:
#### Harry Potter Deathly Hallows Ch 26

with open("gru_hp.txt", 'r') as f:
    file_lines = f.readlines()
for k in file_lines:
    print(k[:-1])

And the drowned the said them to the goblin last of the sword of
And the drogon the goblins shoot of stole the sho sew ollinge to
And the drogon the goblin had note as they were had not seemed t
And the dround his fanter the stool of filling the dragon hear a
And the deepent
and the goblin had not to stole the beard and sh
And the roor sho the wizards through the rook into the cound of
And the rour prase the sward ond the hand of the stord though no
And the rour whes was are allown as the goblin was she triggtt a
And the rour her was sure it a sime his long, and which ware a s
And the roor in a more that they were was the wand had becond th
And the was the goblin said Harry shout them, which ward the swo
And the was the goblin was she still be the singed the serpon al
And the roor in a more that the goblin was the entrout and the w





In [131]:
#### Book of Alma text file
with open("gru_alma.txt", 'r') as f:
    file_lines = f.readlines()
for k in file_lines:
    print(k[:-1])

And and his brethren, who was a praceing against the Lamanites,
And and his brethren, who was a preaching of the Lord doth comma
And and his brethren, who was a preaching of the Lord did not su
And and his brethren, the Lamanites did also be a great strength
And and his brethren and had received and also the same was nor
And an end and tempted upon them into the land of Zarahemla, and
And at the time of the land of Zarahemla, and he should be saved
And at the time of the would have been spoken by the Lamanites,
And at the last day to the land of Zarahemla, and he should be s
And at the last day that he would they see that the Lord had sai
And at the last day that he would that ye should return them of
And at the last day the people of Nephi, that the Lamanites were
And at the seashore, and the Lamanites were strength and began t
And at the seashore, and the Lamanites were strong and his breth
And at the seashore, and they were all the people of Nephi, and 
And at the seashore, and the 

In [132]:
#### Second HP text with more first words
with open("new_hp.txt", 'r') as f:
    file_lines = f.readlines()
for k in file_lines:
    print(k[:-1])

And the trough and see the drow of the sword and the goblins and
He made to dispered and the ground her and Hermione had the sti
Then goblins and spent and seamen the sword and the gragon into t
The goblin a shill seamed the goblin and raver and the was shous
And the trough and see the drow of the sword and the goblins and
He made to dispered and the ground her and Hermione had the sti
Then goblins and spent and seamen the sword and the gragon into t
The goblin a shill seamed the goblin and raver and the was shous
And the trough and seeped the stood there with a thing with a th
He made to and with the sword and the goblin and the sward a sh
Then goblins and the still the ston that the goblin and the sward
The goblin a shill wet of the sword the could her was she stood 
And the trough and seeped the stood there with a thing with a th
He made to and with the sword and the goblin and the sward a sh
Then goblins and the still the ston that the goblin and the sward
The goblin a shill wet of